# Generate Music With Diffusion Models
In this notebook we will implement a Diffusion Probabilistic Model to generate music from raw waveform.\
A Diffusion Model learns to invert a Markov process that progressively adds gaussian noise to the signal until the signal is completely destroyed. As shown in the figure.

<center><img src="./images/diff_model.png"></img></center>

The forward process $q(x_t | x_{t-1})$ is fixed, and it can be parametrized as:

$$
q(x_t|x_{t-1}) = N(x_t;\sqrt{1-\beta_t}x_{t-1},\beta_t I)
$$
Where $\beta_1$,...,$\beta_t$ are givern by a fixed variance schedule (in our case the betas are set to be linearly increasing constants from $10^{-4}$ to 0.02 as in [1]).

If the steps are small enough also the backward process is a gaussian process of the form:

$$
p(x_{t-1}|x_t) = N(x_{t-1},\mu_\theta(x_t,t),\Sigma_\theta(x_t,t))
$$
Where $\mu_\theta$ and $\Sigma_\theta$ are parametric functions with parameters $\theta$.

Diffusion Models are latent variable models of the form:
$$ p_\theta(x_0) = \int p_\theta(x_{0:T}) dx_{1:T}$$
where $x_1$,...,$x_T$ are latents with the same dimensionality of the data $x_0 \sim q(x_0)$.
However the distribution $p_\theta(x_0)$ cannot be optimized directly maximizing the log-lokelihood for the intractability of the integral. Hence we optimize the model maximizing the ELBO:
$$
E_{x_{1:T}}[\log{\frac{p_\theta(x_{0:T})}{p(x_{1:T}|x_0)}}]
$$

The maximization of the ELBO can be reduced to the minimization of a loss that compares directly the learned backward process $p_\theta(x_{t-1}|x_t)$ with the posterior $q(x_{t-1}|x_t,x_0) = N(x_{t-1};\tilde{\mu_t}(x_t,x_0),\tilde{\beta}_tI)$, that is tractable when conditioned on $x_0$. 

We adopted the same reparametrization introduced in [1], according to which:
$$
\tilde{\mu_t}(x_t,x_0) = \frac{\sqrt{\bar{\alpha}_{t-1}}\beta_t}{1-\bar{\alpha}_t}x_0 + \frac{\sqrt{\alpha_t}(1-\bar{\alpha}_{t-1})}{1-\bar{\alpha}_t}x_t
$$
$$
\tilde{\beta}_t = \frac{1-\bar{\alpha}_{t-1}}{1-\bar{\alpha}_t}\beta_t
$$
$$
\bar{\alpha_t} = \prod_{s=1}^t \alpha_s
$$
$$
\alpha_t = 1-\beta_t
$$


In [1]:
import tensorflow as tf
from params import params
from data.dataset import get_wav,get_unlabelled_dataset
import math
import subprocess
import os
from network.model import DiffWaveNet
from diffusion.diffusion_process import train,backward_process

2022-08-24 17:11:59.392813: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
ckpt_path = "/media/giorgio/TOSHIBA EXT/ckpt_10"
train_step = 1000000

## Dataset
To train the model we used a small portion of the <a href="https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification">GTZAN</a> dataset (about 40 tracks), because training the model on the whole dataset was too expensive. The GTZAN dataset contains 1000 traks categorized in 10 genres, even though the categorization is not used in this project. 
### Preprocessing
Each sample is a mono audio file (one channel) that is 30 seconds long with a sampling rate of 22.5 kHz. This implies that each sample is a tensor with shape (675000,1). In particular the sequence length was too long to efficiently train the neural network that will be described in the next section. For this reason each audio file was downsampled by a factor 3. In order to prevent aliasing, before the downsampling, the signal was passed through a low pass filter, that was implemented from scratch in order to be fully integrated in the data pipeline. The low pass filter is implemented by the method `data.dataset.low_pass(signal,fc,npoints=128)` that takes as input a signal with shape (seq_len,n_channels) and the cutoff frequency fc and it returns the same signal without all the frequencies component that are above the cutoff. In order to further reduce the number of samples each downsampled audiofile was split into 6 samples.
The data pipeline is implemented by the method `data.dataset.get_unlabelled_dataset(bs=None,nsplits=1,downsample=1)`, while the values of the parameters used for training are stored in the file `params.py`.
In order to train the model efficiently the dataset was cached into the `dataset/unlabelled_10` directory. 

In [3]:
# if the preprocessed dataset does not exist, then
# create it
if not os.path.exists("dataset/unlabelled_10"):
    ds = get_unlabelled_dataset(params["BS"],nsplits=params["NSPLITS"],downsample=params["DOWNSAMPLE"]).take(10)
    tf.data.experimental.save(
    ds, "./dataset/unlabelled_10")

In [4]:
# set this counter to the number of batches (4 samples each)
# you want to listen
count = 1

ds = tf.data.experimental.load("./dataset/unlabelled_10",tf.TensorSpec(shape=(params["BS"],math.ceil((params["NSAMPLES"]//params["NSPLITS"])/params["DOWNSAMPLE"]),1), dtype=tf.float32))
for curr,x in enumerate(ds):
    if curr >= count:
        break
    for i in range(x.shape[0]):
        wav = get_wav(x[i],params["SR"]//params["DOWNSAMPLE"])
        subprocess.run(["ffplay","-"],input=wav.numpy())


ffplay version 4.2.7-0ubuntu0.1 Copyright (c) 2003-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

ffplay version 4.2.7-0ubuntu0.1 Copyright (c) 2003-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

ffplay version 4.2.7-0ubuntu0.1 Copyright (c) 2003-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

ffplay version 4.2.7-0ubuntu0.1 Copyright (c) 2003-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

## Model
In the previous section we stated that the maximization of the ELBO can be reduced to the minimization of a loss that compares directly the learned backward process $p_\theta(x_{t-1}|x_t)$ with the posterior $q(x_{t-1}|x_t,x_0)$. 
In particular if we recall that 
$$
p_\theta(x_{t-1}|x_t) = N(x_{t-1};\mu_\theta(x_t,t),\sigma^2_tI)
$$
(where we simplified the previous formulation assuming that $\Sigma_\theta(x_t,t) = \sigma^2_tI$ with $\sigma_t^2 = \beta_t$) and
$$
q(x_{t-1}|x_t,x_0) = N(x_{t-1};\tilde{\mu_t}(x_t,x_0),\tilde{\beta}_tI)
$$
then the maximization of the ELBO can be reduced to the minimization of:
$$
E_{t\sim Uniform(1,T)\land x_t\sim q}[\frac{1}{2\sigma^2_t}||\tilde{\mu}_t(x_t,x_0)-\mu_\theta(x_t,t)||^2]
$$
Moreover, it can be shown that by reparametrizing 
$$
x_t \sim q(x_t|x_0) = N(x_t;\sqrt{\bar{\alpha_t}}x_0,(1-\bar{\alpha_t})I)
$$
with 
$$
x_t(x_0,\epsilon) = \sqrt{\bar{\alpha_t}}x_0 + \sqrt{1-\bar{\alpha_t}}\epsilon, \quad\epsilon\sim N(0,I)
$$
than, the loss can be formulated as:
$$
L = || \epsilon - \epsilon_\theta(\sqrt{\bar{\alpha_t}}x_0 + \sqrt{1-\bar{\alpha_t}}\epsilon,t)||^2
$$


At the end of the day, what we need to implement is a neural network $\epsilon_\theta$ that takes as input a sample at any given step t of the forward process $x_t \sim q(x_t|x_0)$ along with the positional encoding of the step t and predicts the gaussian noise $\epsilon$ (that has the same dimensionality as $x_t$).

The positional encoding is implemented by the function `diffusion.positional_encoder.encode(pos,dmodel,n=10000)` as described in [2] while the neural network is an implementation of the DiffWave network presented in [3] for speech sinthesys.
The model architecture is represented in the figure:
<center><img src="./images/arch.png" width=600px></img></center>

The model was trained with the algorithm in the figure, as in [1]:

<center><img src="./images/train_alg.png" width=500px></img></center>

## Generation
After training, the model can be used to generate new samples from gaussian noise. In particular we sample $x_T \sim N(0,I)$ and then we repeat the backward process for the fixed number of steps. This process consists in sampling $x_{t-1} \sim p_\theta(x_{t-1}|x_t$ starting from x_T and repeating untill x_0 is reached. For simplicity, instead of sampling directly $x_{t-1}$ from $p_\theta(x_{t-1}|x_t)$ we can use the same reparametrization we introduced before and compute:
$$
x_{t-1} = \frac{1}{\sqrt{\alpha_t}}(x_t - \frac{\beta_t}{\sqrt{1-\bar{\alpha_t}}}\epsilon_\theta(x_t,t)) + \sigma_t z
$$
with 
$$
z \sim N(0,1)
$$
The algorithm is illustrated in the following figure:
<center><img src="./images/sampl_alg.png" width=500px></img></center>

The sampling algorithm is implemented by the function `diffusion.diffusion_process.backward_process(model,shape,diffusion_steps,return_sequence=False,step_emb_dim:int=128)` that takes as input the neural network, the shape of the sample, and the number of diffusion steps, if return_sequence is set to true the function returns all the intermediate samples $x_t$ with $t \in [T-1, T-2, ...,0]$. It is important to note that the neural network is composed of convolutional layers, with the exception of the branch that takes as input the positional embedding, which shape does not depend on the sequence length. This implies that the network can generate samples with arbitrary length, independently of the sequence length used for training. For this reason in the `backward_process` function we can specify the shape of the sample.


## Results
In order to test the correctness of the implementation, the model was first trained on a single batch (four samples) for 1 Million steps. The loss is shown in the following figure:

<center><img src="./images/test_loss.png" width=600px></img></center>

The weights were saved under the directory `test/model_test/`.



In [5]:
NSAMPLES = 660000
test_net = DiffWaveNet(10,64,3)
test_net.load_weights(f"test/model_test/diff_wave_net")
x_0_gen = backward_process(test_net,(1,NSAMPLES,1),params["DIFF_STEPS"])
wav = get_wav(x_0_gen[0],params["SR"]//params["DOWNSAMPLE"])
subprocess.run(["ffplay","-"],input=wav.numpy())

2022-08-24 17:12:15.108371: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1689600000 exceeds 10% of free system memory.
2022-08-24 17:12:20.597816: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1689600000 exceeds 10% of free system memory.
2022-08-24 17:12:26.074132: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1689600000 exceeds 10% of free system memory.
2022-08-24 17:12:31.544135: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1689600000 exceeds 10% of free system memory.
2022-08-24 17:12:37.025502: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1689600000 exceeds 10% of free system memory.
ffplay version 4.2.7-0ubuntu0.1 Copyright (c) 2003-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=a

CompletedProcess(args=['ffplay', '-'], returncode=0)

In [6]:
net = DiffWaveNet(params["DEPTH"],params["CHANNELS"],params["KERNEL_SIZE"])
params["MODEL_NAME"] = net.name
if not os.path.exists(ckpt_path):
    train(
        ds,
        params["DIFF_STEPS"],
        net,
        tf.keras.optimizers.Adam(learning_rate=2*10**-4),
        net.name,
        ckpt_path=ckpt_path,
        print_every=100,
        save_every=1000,
        resume=False,
        resume_ckpt=None
    )

net.load_weights(os.path.join(ckpt_path,f"__step_{train_step}__diff_wave_net"))

## Bibliography
- [1] Ho, Jonathan, Ajay Jain, and Pieter Abbeel. "Denoising diffusion probabilistic models." Advances in Neural Information Processing Systems 33 (2020): 6840-6851.
- [2] Vaswani, Ashish, et al. "Attention is all you need." Advances in neural information processing systems 30 (2017).
- [3] Kong, Zhifeng, et al. "Diffwave: A versatile diffusion model for audio synthesis." arXiv preprint arXiv:2009.09761 (2020).